In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import autosklearn.classification as automl
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy import sparse
from sklearn.preprocessing import OneHotEncoder
from autosklearn.experimental.askl2 import AutoSklearn2Classifier
from autosklearn.regression import AutoSklearnRegressor

/usr/lib/python3/dist-packages/pyparsing.py:3190: FutureWarning: Possible set intersection at position 3
  self.re = re.compile(self.reString)


# Initalize The Data Structures

In [2]:
cls = AutoSklearn2Classifier(
    time_left_for_this_task=60*30,
    per_run_time_limit=60*30,
    ensemble_nbest=1,
    n_jobs=12
)

In [3]:
BATCH_SIZE = 20000
TEST_SIZE = 2400
CHUNKSIZE = BATCH_SIZE + TEST_SIZE
dataiter = pd.read_csv('game_data_public.KHM.PremierDraft.csv',chunksize=CHUNKSIZE)

# Prep a new Sample


In [4]:
def nextSample():
    data = next(dataiter)
    decks = data.loc[:,data.columns.str.contains("deck")]
    decks = decks.drop(columns=['deck_Forest', 'deck_Island', 'deck_Mountain', 'deck_Plains',
           'deck_Swamp','deck_Snow-Covered Forest', 'deck_Snow-Covered Island',
           'deck_Snow-Covered Mountain','deck_Snow-Covered Swamp','deck_Snow-Covered Plains'])
    X = decks.loc[:]
    X = X>0
    X = np.array(X).astype(int)
    #X = sparse.csr_matrix(X)
    y = data.loc[:,"won"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE/CHUNKSIZE, random_state=42)
    cls.fit(X_train, y_train)
    return (X_train, X_test, y_train, y_test)

# Run a AutoML iteration

In [5]:
for i in range(14):
    X_train, X_test, y_train, y_test = nextSample()
    y_hat = cls.predict(X_test)
    print("Accuracy score:", accuracy_score(y_test, y_hat))

INFO:numexpr.utils:Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
INFO:numexpr.utils:NumExpr defaulting to 8 threads.


Using no backup selector
Accuracy score: 0.54375
Using no backup selector
Accuracy score: 0.55
Using no backup selector
Accuracy score: 0.5554166666666667
Using no backup selector
Accuracy score: 0.5441666666666667
Using no backup selector
Accuracy score: 0.54125
Using no backup selector


/usr/local/lib/python3.9/dist-packages/smac/tae/dask_runner.py:148: UserWarning: No workers are available. This could mean workers crashedWaiting for new workers...
  warnings.warn("No workers are available. This could mean workers crashed"


OSError: Timed out trying to connect to inproc://172.26.126.106/7555/271 after 10 s

# Show Statistics

In [ ]:
print(cls.show_models())

In [ ]:
cls.sprint_statistics()

In [ ]:
print(cls.cv_results_)